In [8]:
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.units import unit

from openff.interchange import Interchange
from openff.interchange.drivers import get_gromacs_energies, get_openmm_energies
import multiprocessing
import tqdm
import numpy as np
import sys
import openmm
from openff.units.openmm import from_openmm
import os
# suppress stereochemistry warnings
import logging
from rdkit import Chem
import itertools

from openff.qcsubmit.results import TorsionDriveResultCollection, OptimizationResultCollection
import logging
logging.getLogger("openff").setLevel(logging.ERROR)

from openff.qcsubmit.results.filters import SMARTSFilter

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolTransforms

import MDAnalysis as mda

In [2]:
def minimize_energy(mol,ff):
    topo = mol.to_topology()
    interchange = Interchange.from_smirnoff(force_field=ff, topology=topo,allow_nonintegral_charges=True)#,charge_from_molecules=[mol]) #,charge_from_molecules=[mol])
    integrator = openmm.VerletIntegrator(1 * openmm.unit.femtoseconds)
    simulation = interchange.to_openmm_simulation(integrator)

    # We'll store energies in two lists
    initial_energies = []
    minimized_energies = []

    # And minimized conformers in a second molecule
    minimized_molecule = Molecule.from_topology(topo)
    minimized_molecule.conformers.clear()

    conformer = mol.conformers[0]
    # Tell the OpenMM Simulation the positions of this conformer
    simulation.context.setPositions(conformer.to_openmm())

    # Keep a record of the initial energy
    initial_energies.append(
        simulation.context.getState(getEnergy=True).getPotentialEnergy()
    )

    # Perform the minimization
    simulation.minimizeEnergy()#tolerance=Quantity(value=0.0026255, unit=kilojoule/mole))#tolerance=5e-9)

    # Record minimized energy and positions
    min_state = simulation.context.getState(getEnergy=True, getPositions=True)

    minimized_energies.append(min_state.getPotentialEnergy())
    minimized_molecule.add_conformer(from_openmm(min_state.getPositions()))
    return initial_energies,minimized_energies,minimized_molecule


# Molecule 1

In [3]:
# test_mol = industry_benchmark_r3_rm[15][1]
# test_mol

In [4]:
# industry_benchmark_r3_rm[15]

In [5]:
# test_mol.to_file('qcaid_37008138.sdf',file_format='sdf')

In [6]:
test_mol = Molecule.from_file("qcaid_37008138.sdf", "SDF")

In [9]:
ie_sage200_qm,me_sage200_qm,min_mol_sage200_qm = minimize_energy(test_mol,ForceField('openff_unconstrained-2.0.0.offxml'))
u_sage200_1 = mda.Universe(min_mol_sage200_qm.to_rdkit())
u_sage200_1.atoms.write("sage-200_qcaid_37008138_opt.pdb")

/Users/lily/micromamba/envs/yammbs/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/Users/lily/micromamba/envs/yammbs/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/lily/micromamba/envs/yammbs/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'resnames' Using default value of 'UNK'
  warnings.warn("Found no information for attr: '{}'"
/Users/lily/micromamba/envs/yammbs/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/lily/micromamba/envs/yammbs/lib/python3.10/site-packag

In [10]:
ie_sage210_qm,me_sage210_qm,min_mol_sage210_qm = minimize_energy(test_mol,ForceField('openff_unconstrained-2.1.0.offxml'))
u_sage210_1 = mda.Universe(min_mol_sage210_qm.to_rdkit())
u_sage210_1.atoms.write("sage-210_qcaid_37008138_opt.pdb")

In [12]:
ie_sage220_qm,me_sage220_qm,min_mol_sage220_qm = minimize_energy(test_mol,ForceField('../../../openff_unconstrained-2.2.0.offxml',allow_cosmetic_attributes=True))
u_sage220_1 = mda.Universe(min_mol_sage220_qm.to_rdkit())
u_sage220_1.atoms.write("sage-220_qcaid_37008138_opt.pdb")

In [13]:
ie_sage221_qm,me_sage221_qm,min_mol_sage221_qm = minimize_energy(test_mol,ForceField('../../../openff_unconstrained-2.2.1-rc1.offxml',allow_cosmetic_attributes=True))
u_sage221_1 = mda.Universe(min_mol_sage221_qm.to_rdkit())
u_sage221_1.atoms.write("sage-221_qcaid_37008138_opt.pdb")

In [31]:
u_qm_min = mda.Universe(test_mol.to_rdkit())
u_qm_min.atoms.write("qm_qcaid_37008138_opt.pdb")

In [14]:
min_mol_sage220_qm.to_file('qcaid_37008138_optsage220.sdf',file_format='sdf')
min_mol_sage221_qm.to_file('qcaid_37008138_optsage221.sdf',file_format='sdf')

In [15]:
opt_sage200 = min_mol_sage200_qm.to_rdkit()
opt_sage200_conf = opt_sage200.GetConformer()

In [16]:
opt_sage210 = min_mol_sage210_qm.to_rdkit()
opt_sage210_conf = opt_sage210.GetConformer()

In [17]:
opt_sage220 = min_mol_sage220_qm.to_rdkit()
opt_sage220_conf = opt_sage220.GetConformer()

In [18]:
opt_sage221 = min_mol_sage221_qm.to_rdkit()
opt_sage221_conf = opt_sage221.GetConformer()

In [19]:
qm_rdkit = test_mol.to_rdkit()
qm_conf = qm_rdkit.GetConformer()

#### Geometries

In [20]:
o = [a[0] for a in test_mol.chemical_environment_matches('[#8;r3:1]')][0]
c1,c2 = [a[0] for a in test_mol.chemical_environment_matches('[#6;r3:1]')]
print(o,c1,c2)

12 7 8


In [21]:
print(' O-C1: ', Chem.rdMolTransforms.GetBondLength(opt_sage200_conf,o,c1) )
print(' O-C2: ', Chem.rdMolTransforms.GetBondLength(opt_sage200_conf,o,c2) )
print('C1-C2: ', Chem.rdMolTransforms.GetBondLength(opt_sage200_conf,c1,c2) )

 O-C1:  1.4150285595524377
 O-C2:  1.41733725591812
C1-C2:  1.5632267228927386


In [22]:
print(' O-C1: ', Chem.rdMolTransforms.GetBondLength(opt_sage210_conf,o,c1) )
print(' O-C2: ', Chem.rdMolTransforms.GetBondLength(opt_sage210_conf,o,c2) )
print('C1-C2: ', Chem.rdMolTransforms.GetBondLength(opt_sage210_conf,c1,c2) )

 O-C1:  1.2873620226815634
 O-C2:  1.2950843100145535
C1-C2:  1.7419940286102387


In [23]:
print(' O-C1: ', Chem.rdMolTransforms.GetBondLength(opt_sage220_conf,o,c1) )
print(' O-C2: ', Chem.rdMolTransforms.GetBondLength(opt_sage220_conf,o,c2) )
print('C1-C2: ', Chem.rdMolTransforms.GetBondLength(opt_sage220_conf,c1,c2) )

 O-C1:  1.4490246050548203
 O-C2:  1.4532513106199694
C1-C2:  1.5282125235523683


In [24]:
print(' O-C1: ', Chem.rdMolTransforms.GetBondLength(opt_sage221_conf,o,c1) )
print(' O-C2: ', Chem.rdMolTransforms.GetBondLength(opt_sage221_conf,o,c2) )
print('C1-C2: ', Chem.rdMolTransforms.GetBondLength(opt_sage221_conf,c1,c2) )

 O-C1:  1.4486861393740798
 O-C2:  1.4534532804075349
C1-C2:  1.5278395203142217


In [25]:
print(' O-C1: ', Chem.rdMolTransforms.GetBondLength(qm_conf,o,c1) )
print(' O-C2: ', Chem.rdMolTransforms.GetBondLength(qm_conf,o,c2) )
print('C1-C2: ', Chem.rdMolTransforms.GetBondLength(qm_conf,c1,c2) )

 O-C1:  1.4307177739703938
 O-C2:  1.430787564044903
C1-C2:  1.4904549684441248


In [26]:
print('C1 - O - C2:',Chem.rdMolTransforms.GetAngleDeg(opt_sage200_conf,c1,o,c2)) 
print('C1 - C2 - O:',Chem.rdMolTransforms.GetAngleDeg(opt_sage200_conf,c1,c2,o)) 
print('O - C1 - C2:',Chem.rdMolTransforms.GetAngleDeg(opt_sage200_conf,o,c1,c2)) 


C1 - O - C2: 66.99699621656426
C1 - C2 - O: 56.43093813148617
O - C1 - C2: 56.57206565194957


In [27]:
print('C1 - O - C2:',Chem.rdMolTransforms.GetAngleDeg(opt_sage210_conf,c1,o,c2)) 
print('C1 - C2 - O:',Chem.rdMolTransforms.GetAngleDeg(opt_sage210_conf,c1,c2,o)) 
print('O - C1 - C2:',Chem.rdMolTransforms.GetAngleDeg(opt_sage210_conf,o,c1,c2)) 


C1 - O - C2: 84.83816084511845
C1 - C2 - O: 47.393413723283665
O - C1 - C2: 47.768425431597876


In [28]:
print('C1 - O - C2:',Chem.rdMolTransforms.GetAngleDeg(opt_sage220_conf,c1,o,c2)) 
print('C1 - C2 - O:',Chem.rdMolTransforms.GetAngleDeg(opt_sage220_conf,c1,c2,o)) 
print('O - C1 - C2:',Chem.rdMolTransforms.GetAngleDeg(opt_sage220_conf,o,c1,c2)) 


C1 - O - C2: 63.545987167644746
C1 - C2 - O: 58.092286369377696
O - C1 - C2: 58.36172646297757


In [29]:
print('C1 - O - C2:',Chem.rdMolTransforms.GetAngleDeg(opt_sage221_conf,c1,o,c2)) 
print('C1 - C2 - O:',Chem.rdMolTransforms.GetAngleDeg(opt_sage221_conf,c1,c2,o)) 
print('O - C1 - C2:',Chem.rdMolTransforms.GetAngleDeg(opt_sage221_conf,o,c1,c2)) 


C1 - O - C2: 63.531948063796975
C1 - C2 - O: 58.08203166709584
O - C1 - C2: 58.38602026910717


In [30]:
print('C1 - O - C2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf,c1,o,c2)) 
print('C1 - C2 - O:',Chem.rdMolTransforms.GetAngleDeg(qm_conf,c1,c2,o)) 
print('O - C1 - C2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf,o,c1,c2)) 


C1 - O - C2: 62.78044254406454
C1 - C2 - O: 58.60748853517811
O - C1 - C2: 58.61206892075735
